In [1]:
import pandas as pd

import itertools
import random

from joblib import Parallel, delayed, dump
import multiprocessing
import logging
import sys
import warnings
import time

from sklearn.pipeline import Pipeline
from sklearn.metrics import get_scorer

**La siguiente celda sirve para ver la cantidad de núcleos en la CPU**, para que sepan en cuantos trabajos dividir las tareas. La idea es realizar tantos procesos paralelos como núcleos tenga nuestra CPU (el dataset es chico, asique dudo que haya problemas con la RAM, pero si nos quedamos sin RAM, hay que paralelizar en menos procesos que la cantidad de núcleos, o no hacerlo en absoluto). Esto es para que podamos hacer búsquedas más exhaustivas en menos tiempo. Entiendo que scikit-learn no aprovecha GPU (según chat GPT), pero XGBoost si 👀 (@fede, pero si lo querés aprovechar tendrías que cambiar la función, e indicarlo en un argumento al instanciar la clase$^1$).

$^1$ no entiendo mucho de GPUs, capaz vos entendés más pero comparto lo que leí por si sirve.

In [2]:
print(f"Available CPU cores: {multiprocessing.cpu_count()}")

Available CPU cores: 4


## Función para entrenar y evaluar

In [3]:
def optimizador_hiperparametros(nombre, config, X_train, y_train, X_val, y_val, nombre_dataset, metrica,
                                busqueda='random', random_state=42, n_iter=20, n_jobs=1):
    """
    Función para optimiar los hiperparámetros de un modelo dado. Se entrenan varios modelos sobre el conjunto
    entrenamiento y luego se comparan sus métricas sobre el conjunto de validación (no se hace cross
    validation).
    Se tiene opción de paralelizar los entrenamientos.
    
    Paámetros:
    ----------
        nombre : str
            nombre con el que se identifica al modelo
            
        config : dict
            Diccionario con la configuración del modelo. Debe tener los siguientes pares de clave - valor:
                - 'modelo' : Instancia básica del modelo a probar. Debe ser un objeto de sklearn, xgboost o
                    alguna librería afín.
                'params' : diccionario. Sus claves son los hiperparámetros (dados en la documentación de la
                    clase de clasificador / regresor que se utilice) y sus valores corresponden a listas con
                    los rangos o valores a probar.
                    
        X_train : pandas.DataFrame
            Dataframe del conjunto entrenamiento (sólo las muestras).
        y_train : pandas.DataFrame | pandas.Serie
            Dataframe con los valores de la variable objetivo del conjunto de entrenamiento.
        X_val : pandas.DataFrame
            Dataframe con las muestras del conjunto de validación.
        y_val : pandas.DataFrame | pandas.Serie
            Dataframe con los valores de la variable objetivo del conjunto de validación.
            
        nombre_dataset : str
            Nombre con el que se identifica al dataset. Se utilizará como sufijo para identificar con qué
            dataset que entrenado el modelo.
            
        metrica : str
            Métrica a utilizar. Consultar opciones válidas en
            https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-string-names.
            
        busqueda : str, default='random'
            La forma en que se optimizarán los hiperparámetros. Opciones:
                - 'random' : random search
                - 'grid' : grid search
                
        random_state : int, default=42
            Valor semilla para la elección aleatoria en caso de usar random search.
            Se fija para reproducibilidad.
            
        n_iter : int, default=20
            Cantidad de combinaciones de valores de hiperparámetros a probar en caso de que se use un random
            search.
            
        n_jobs : int, default=1
            Cantidad de trabajos a correr en paralelo. Se sugiere que tenga como valor la cantidad de núcleos
            de CPU (no me maten si estoy diciendo chanchadas, así lo entendí).
            
    Devuelve:
    ---------
    """
    start_time = time.time()
    print(f"🚀 Iniciando entrenamiento para: {nombre}")
    
    scorer = get_scorer(metrica)
    
    param_space = config["params"]
    all_param_combinations = list(itertools.product(*param_space.values()))
    param_keys = list(param_space.keys())
    
    total_combinations = 1
    for v in param_space.values():
        total_combinations *= len(v)
    n_iter_ajustado = min(8, total_combinations)

    # Random o grid
    if busqueda == 'grid':
        candidates = all_param_combinations
    elif busqueda == 'random':
        random.seed(random_state)
        candidates = random.sample(all_param_combinations,
                                   min(n_iter, len(all_param_combinations)))
    else:
        raise ValueError("busqueda debe ser 'random' o 'grid'.")
        
    # Función auxiliar para entrenar y evaluar un modelo
    def _fit_and_score(params):
        param_dict = dict(zip(param_keys, params))
        pipeline = Pipeline([('clf', config["modelo"].set_params(**param_dict))])
        pipeline.fit(X_train, y_train)
        score = -scorer(pipeline, X_val, y_val)
        return param_dict, score, pipeline
    
    # Paralelizar entrenamiento
    results = Parallel(n_jobs=n_jobs)(
        delayed(_fit_and_score)(params) for params in candidates
    )
        
    # Seleccionar el mejor
    best_params, best_score, best_model = max(results, key=lambda x: x[1])
            
    print(f"✅ {nombre}: búsqueda finalizada.")
    print(f"🏆 Mejor score en validación: {best_score:.4f}")
    print(f"📌 Mejores hiperparámetros: {best_params}")

    elapsed = time.time() - start_time
    print(f"⏱️ Tiempo entrenando {nombre}: {elapsed:.2f} segundos")

    # Guardar modelo
    dump(best_model, f"./{nombre}_{nombre_dataset}.joblib")

    return best_model, best_params, best_score

## Importo datasets

In [4]:
## Importamos los datasets A

# Etiquetas
y_A_train = pd.read_csv("./A_sets/y_train_inicial.csv")
y_A_test = pd.read_csv("./A_sets/y_test_inicial.csv")
# Datos escaleados
X_A_train_scaled = pd.read_csv("./A_sets/x_A_train_norm.csv")
X_A_test_scaled = pd.read_csv("./A_sets/x_A_test_norm.csv")
# Datos estandarizados
X_A_train_std = pd.read_csv("./A_sets/x_A_train_std.csv")
X_A_test_std = pd.read_csv("./A_sets/x_A_test_std.csv")

In [5]:
## Importamos los datasets B

# Etiquetas
y_B_train = pd.read_csv("./B_sets/y_train_inicial.csv")
y_B_test = pd.read_csv("./B_sets/y_test_inicial.csv")
# Datos escaleados
X_B_train_scaled = pd.read_csv("./B_sets/x_B_train_norm.csv")
X_B_test_scaled = pd.read_csv("./B_sets/x_B_test_norm.csv")
# Datos estandarizados
X_B_train_std = pd.read_csv("./B_sets/x_B_train_std.csv")
X_B_test_std = pd.read_csv("./B_sets/x_B_test_std.csv")

In [6]:
## Defino diccinario para iterar sobre los datasets

datasets_dict = {
    'A_scaled' : {'x_train':X_A_train_scaled, 'y_train':y_A_train, 'x_test':X_A_test_scaled, 'y_test':y_A_test},
    'A_std' : {'x_train':X_A_train_std, 'y_train':y_A_train, 'x_test':X_A_test_std, 'y_test':y_A_test},
    'B_scaled' : {'x_train':X_B_train_scaled, 'y_train':y_B_train, 'x_test':X_B_test_scaled, 'y_test':y_B_test},
    'B_std' : {'x_train':X_B_train_std, 'y_train':y_B_train, 'x_test':X_B_test_std, 'y_test':y_B_test},
}

## Ejemplos de uso

### LASSO

In [7]:
from sklearn.linear_model import Lasso
import numpy as np

lasso_config = {
    "modelo": Lasso(random_state=42),
    "params": {
        "alpha": np.logspace(-4, 4, num=9, base=10),
        "fit_intercept": [True, False],
        "positive": [True, False],
        "selection": ['cyclic', 'random']
    }
}

In [9]:
resultados = {}

for set_conjuntos, set_dict in datasets_dict.items():
    print(f'====================\nDataset {set_conjuntos}\n====================')
    best_model, best_params, best_score = optimizador_hiperparametros(nombre='LASSO',
                                                                      config=lasso_config,
                                                                      X_train=set_dict['x_train'],
                                                                      y_train=set_dict['y_train'],
                                                                      X_val=set_dict['x_test'],
                                                                      y_val=set_dict['y_test'],
                                                                      nombre_dataset=set_conjuntos,
                                                                      metrica='neg_mean_absolute_error',
                                                                      busqueda='random',
                                                                      random_state=42,
                                                                      n_iter=20,
                                                                      n_jobs=4)
    resultados[set_conjuntos] = {'mejor_modelo' : best_model,
                                 'mejores_params' : best_params,
                                 'mejor_score' : best_score
                                }
    print("\n")

Dataset A_scaled
🚀 Iniciando entrenamiento para: LASSO
✅ LASSO: búsqueda finalizada.
🏆 Mejor score en validación: 9.1143
📌 Mejores hiperparámetros: {'alpha': np.float64(10000.0), 'fit_intercept': False, 'positive': False, 'selection': 'random'}
⏱️ Tiempo entrenando LASSO: 0.14 segundos


Dataset A_std
🚀 Iniciando entrenamiento para: LASSO


/home/edu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.700e+02, tolerance: 8.195e+01
  model = cd_fast.enet_coordinate_descent(


✅ LASSO: búsqueda finalizada.
🏆 Mejor score en validación: 9.1143
📌 Mejores hiperparámetros: {'alpha': np.float64(10000.0), 'fit_intercept': False, 'positive': False, 'selection': 'random'}
⏱️ Tiempo entrenando LASSO: 0.13 segundos


Dataset B_scaled
🚀 Iniciando entrenamiento para: LASSO
✅ LASSO: búsqueda finalizada.
🏆 Mejor score en validación: 9.1143
📌 Mejores hiperparámetros: {'alpha': np.float64(10000.0), 'fit_intercept': False, 'positive': False, 'selection': 'random'}
⏱️ Tiempo entrenando LASSO: 0.12 segundos


Dataset B_std
🚀 Iniciando entrenamiento para: LASSO
✅ LASSO: búsqueda finalizada.
🏆 Mejor score en validación: 9.1143
📌 Mejores hiperparámetros: {'alpha': np.float64(10000.0), 'fit_intercept': False, 'positive': False, 'selection': 'random'}
⏱️ Tiempo entrenando LASSO: 0.11 segundos




### Regresión polinomial

In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np

polynomial_reg_config = {
    "modelo": Pipeline([
        ("poly", PolynomialFeatures()),
        ("linreg", LinearRegression())
    ]),
    "params": {
        "poly__degree": np.arange(1, 4),
        "poly__interaction_only": [True, False],
        "poly__include_bias": [True, False]
    }
}


In [13]:
resultados = {}

for set_conjuntos, set_dict in datasets_dict.items():
    best_model, best_params, best_score = optimizador_hiperparametros(nombre='regresor_polinomico',
                                                                      config=polynomial_reg_config,
                                                                      X_train=set_dict['x_train'],
                                                                      y_train=set_dict['y_train'],
                                                                      X_val=set_dict['x_test'],
                                                                      y_val=set_dict['y_test'],
                                                                      nombre_dataset=set_conjuntos,
                                                                      metrica='neg_mean_absolute_error',
                                                                      busqueda='random',
                                                                      random_state=42,
                                                                      n_iter=20,
                                                                      n_jobs=4)
    resultados[set_conjuntos] = {'mejor_modelo' : best_model,
                                 'mejores_params' : best_params,
                                 'mejor_score' : best_score
                                }
    print(f'Para dataset {set_conjuntos}, tenemos:')
    print(f'\tMejores parámetros:\n\t{best_params}')
    print(f'\tMejor RMSE: {best_score}\n')
    print('-----------------------------------')

🚀 Iniciando entrenamiento para: regresor_polinomico
✅ regresor_polinomico: búsqueda finalizada.
🏆 Mejor score en validación: 139.3962
📌 Mejores hiperparámetros: {'poly__degree': np.int64(3), 'poly__interaction_only': True, 'poly__include_bias': False}
⏱️ Tiempo entrenando regresor_polinomico: 1.15 segundos
Para dataset A_scaled, tenemos:
	Mejores parámetros:
	{'poly__degree': np.int64(3), 'poly__interaction_only': True, 'poly__include_bias': False}
	Mejor RMSE: 139.39616205184186

-----------------------------------
🚀 Iniciando entrenamiento para: regresor_polinomico
✅ regresor_polinomico: búsqueda finalizada.
🏆 Mejor score en validación: 109.2265
📌 Mejores hiperparámetros: {'poly__degree': np.int64(3), 'poly__interaction_only': False, 'poly__include_bias': False}
⏱️ Tiempo entrenando regresor_polinomico: 1.05 segundos
Para dataset A_std, tenemos:
	Mejores parámetros:
	{'poly__degree': np.int64(3), 'poly__interaction_only': False, 'poly__include_bias': False}
	Mejor RMSE: 109.226482755